<a href="https://colab.research.google.com/github/eugene-h-lin/Android-SlideExpandableListView/blob/master/prototype/EstimateBreak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

載入資料

In [ ]:
def load_data():
  df_history = pd.read_json('data/history.json')
  df_presure = pd.read_json('data/presure.json')
  df_pivot = pd.read_json('data/pivot.json')
  return

定義DataType

In [ ]:
from dataclasses import dataclass, field
from typing import Iterator, List, Optional

import pandas as pd

# ============================
# Data Structures
# ============================

@dataclass(frozen=True)
class StockHistory:
    stock_id: str
    date: pd.Timestamp
    open: float
    high: float
    low: float
    close: float
    transaction: int

@dataclass(frozen=True)
class Pivot:
    stock_id: str
    date: pd.Timestamp
    type: str
    price: float
    level: int
    confirm_date: pd.Timestamp

@dataclass
class Zone:
    stock_id: str
    low: float
    high: float
    confirm_date: pd.Timestamp
    type: str
    expire_date: Optional[pd.Timestamp] = None
    # pivots: List[Pivot] = field(default_factory=list)
    strength: float = 0.0
    level_score: float = 0.0
    hit:int=1

    def contains_price(self, price: float, tolerance=0.02):
        """Check if price is within 2% distance of zone."""
        center = (self.low + self.high) / 2
        return self.high*1.02 >= price and self.low*0.98 <= price # abs(price - center) / center <= tolerance

    def update_boundary(self):
        """Recalculate zone boundaries based on member pivots."""
        prices = [p.price for p in self.pivots]
        self.low = min(prices)
        self.high = max(prices)

    def to_dict(self):
        return self.__dict__

@dataclass
class Zones:
    zones: List[Zone] = field(default_factory=list)

    def __iter__(self) -> Iterator[Zone]:
        """
        定義如何迭代 Zones 物件。
        """
        return iter(self.zones)

    def __len__(self) -> int:
        """
        定義 len(Zones) 的行為。
        """
        return len(self.zones)

    def to_df(self):
        # df: pd.DataFrame
        print(self.zones[0])

        dict_list = [z.to_dict() for z in self.zones]
        print(dict_list[0])

        df = pd.DataFrame(dict_list)
        print(df.iloc[0])

        return df

    def add_zone(self, zone: Zone):
        self.zones.append(zone)

    def get_match_zones(self, type:str, price:int, tolerance=0.01):
        matched = [z for z in self.zones if z.contains_price(price, tolerance) and (not z.expire_date) and (z.type == type)]
        return Zones(zones=matched)

In [ ]:
df_history, df_presure, df_pivot = load_data()

In [ ]:
last_close = df_history.iloc[-1]['close']
last_2_close = df_history.iloc[-2]['close']

# 找出突破的壓力區間
breaked_sr = [presure for presure in df_presure if presure.high < last_close and presure.high >= last_2_close]

# 若有突破，找出前次價格高於今日之後的pivot和history
#  先找出前次價格高於今日的日期
df_history_high = df_history[df_history['high'] > last_close]
d_date = df_history_high.iloc[-1]['date']

#  再找出日期之後的pivot
df_pivot_recent = df_pivot[df_pivot['date'] > d_date]

#  再找出日期之後的history
df_history_recent = df_history[df_history['date'] > d_date]

In [ ]:
def pattern_analysis(df_pivot: pd.DataFrame):
  patterns = []
  top_cnt = 0
  bottom_cnt = 0

  # 判斷有幾個頂，幾個底
  if len(df_pivot) > 0:
     top_cnt = len(df_pivot[df_pivot['type'] == 'high' and df_pivot['level'] > 1])
     bottom_cnt = len(df_pivot[df_pivot['type'] == 'low' and df_pivot['level'] > 1])

  # 如果有超過兩個底，判斷底部型態
  if bottom_cnt >= 2:
    # 判斷底底高或底底低
    check_pivot_trend(df_pivot[df_pivot['type']=='low' and df_pivot['level'] > 1])
    # 判斷是否雙底或三底
    check_bottoms(df_pivot(df_pivot[df_pivot['type']=='low' and df_pivot['level'] > 1]))

  # 如果有超過三個底，判斷頭肩底
  if bottom_cnt >= 3:
    check_head_shelder(df_pivot)

  return

In [ ]:
# 這邊檢查最近的幾個pivot是符合higher 或 lower
def check_pivot_trend(df_bottoms: pd.DataFrame):

  result:str = None
  last_bottom_p = 0
  direction:str = None
  count = 0

  if len(df_bottoms) >= 2:
    for bottom in reversed(df_bottoms):

      if last_bottom_p == 0:
        last_bottom_p == bottom.price
        continue

      if last_bottom_p > bottom.price:
        if(direction is None or direction == 'higher'):
          direction = 'higher'
          count += 1
          last_bottom_p = bottom.price
        else:
          break
      elif last_bottom_p < bottom.price:
        if(direction is None or direction == 'lower'):
          direction = 'lower'
          count += 1
          last_bottom_p = bottom.price
        else:
          break
      else:
        continue

    result = {f"{direction}": count}

  return result

In [ ]:
# 這邊檢查最接近的幾個pivot是在同一水平
def check_bottoms(df_bottoms: pd.DataFrame):
  return


In [ ]:
# 這邊檢查最接近的幾個谷峰是在同一水平
def check_tops(df_tops: pd.DataFrame):
  return


In [ ]:
# 這邊最差要回頸線水平，好一點要回有斜率的頸線
def check_head_shelder(df_pivot: pd.DataFrame):
  return

In [ ]:
# 這邊最差要回頸線水平，好一點要回有斜率的頸線
def check_inv_head_shelder(df_pivot: pd.DataFrame):

  # 1. 先檢查至少要有五個點(五筆資料) >> 沒有就結束

  # 2.1 找頭(最低點)
  # 2.2 檢查頭左右要各有兩個點

  #   2.3.1 如果頭和df_pivot結尾之間還有沒當過右肩的low
  #     2.3.1.1 找右肩(頭和df_pivot結尾之間的下一個最低點)
  #     2.3.1.2 檢查右肩和頭中間至少要有一個點 (沒有就continue)
  #     2.3.2.1 找右頸(頭和右肩中間高於頭和右肩的點中的最高者) (沒有就continue)

  #   2.4.1 如果頭和df_pivot開頭之間還有沒當過左肩的low
  #     2.4.1.1 找左肩(頭和df_pivot開頭之間的下一個最低點)
  #     2.4.1.2 檢查左肩和頭中間至少要有一個點 (沒有就continue)
  #     2.4.2.1 找左頸(頭和左肩中間高於頭和左肩的點中的最高者) (沒有就continue)

  # 2.3 如果頭部右邊有超過2點 >> 找右邊最低點(新的頭)，然後往左找最後一筆價格高於頭部的日期，把df_pivot該日期以後部分的丟進check_inv_head_shelder再CHECK一次

  return